In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.io.wavfile

In [ ]:
# This function extracts the frames
# samples_per_frame and samples_per_skip in seconds

def extract_frame(data, rate, samples_per_frame, samples_per_skip, discard_last = True):
    data_length = data.shape[0]
    frames = []
    current_sample = 0
    while(current_sample <= data_length - samples_per_frame):
        current_frame = data[current_sample : current_sample + samples_per_frame]
        current_sample = current_sample + samples_per_skip
        frames.append(current_frame)
    
    if(not discard_last):
        if current_sample < data_length:    
            last_frame = data[current_sample:]
            frames.append(last_frame)
    
    frames = np.array(frames)
    return frames

# This function applies fft to windowed frames
def fft_frame(frames, fft_length):
    frame_count = frames.shape[0]
    fft_frames = []
    for f in range(frame_count):
        current_frame = frames[f] * np.hamming(frames[f].shape[0])
        fft_frames.append(np.fft.fft(current_frame, fft_length))
    fft_frames = np.array(fft_frames)
    return fft_frames

# This function estimates rough pitch integers for a given pitch interval
def estimate_pitch(frames,samples_per_frame,periods):
    pitch_estimates = []
    for f in frames:
        window = np.hamming(samples_per_frame)
        scaled_frame = np.multiply(np.multiply(window,window),f)
        autocorr = np.correlate(scaled_frame,scaled_frame,'full')
        autocorr_index = np.arange(-samples_per_frame + 1, samples_per_frame)
        theta_p = []
        for p in periods:
            min_k = int((-samples_per_frame + 1)/p)
            max_k = int(samples_per_frame/p)
            k = np.arange(min_k, max_k)
            theta_p.append(p * np.sum(autocorr[k * p + samples_per_frame-1]))
        theta_p = np.array(theta_p)
        max_theta_p = np.argmax(theta_p)
        rough_estimate = periods[max_theta_p]
        pitch_estimates.append(rough_estimate)
    pitch_estimates = np.array(pitch_estimates)
    return pitch_estimates

# This function refines pitch period based on rough integer estimate
def estimate_amplitude(fft_frames,samples_per_frame,pitch_estimates,fft_length):
    Ams = []
    choices = []
    ems = []
    error_sum = []
    refined_pitch_estimates = []
    for i in range(len(pitch_estimates)):
        fft_frame = fft_frames[i]
        rough_pitch = pitch_estimates[i]
        refined_pitches = np.arange(rough_pitch-2, rough_pitch+2+0.2, 0.2)
        
        min_error = np.inf      
        best_Am = []
        best_choices = []
        best_ems = []
        best_pitch = -1
        for refined_pitch in refined_pitches:
            w_0 = 2 * np.pi / refined_pitch
            refined_Ams = []
            refined_choices = []
            refined_ems = []
            
            for m in range(1,int(refined_pitch)):
                center_freq = w_0 * m
                lower_freq = w_0 * (m-1/2)
                upper_freq = w_0 * (m+1/2)
                
                freq_rate = (2*np.pi/fft_length)
                lower_index = int(lower_freq / freq_rate) + 1
                upper_index = min(int(upper_freq / freq_rate), fft_length - 1)
                middle_index = int(np.around(center_freq / freq_rate))
                                  
                Sw = fft_frame[lower_index:upper_index+1]
                norm = np.sum(Sw *np.conj(Sw))
                fft_window = np.fft.fft(np.hamming(samples_per_frame),fft_length)
                                  
                Ew_voiced = np.append(fft_window[fft_length - (middle_index - lower_index):fft_length],
                                      fft_window[0:upper_index - middle_index + 1])
                numer_Am_voiced = Sw * np.conj(Ew_voiced)
                denom_Am_voiced = Ew_voiced * np.conj(Ew_voiced)
                numer_integrand = (np.sum(numer_Am_voiced) - 1/2*(numer_Am_voiced[0]+numer_Am_voiced[-1]))
                denom_integrand = (np.sum(denom_Am_voiced) - 1/2*(denom_Am_voiced[0]+denom_Am_voiced[-1]))
                Am_voiced = numer_integrand / denom_integrand
                em_integrand = (Sw - Am_voiced * Ew_voiced) * np.conj((Sw - Am_voiced * Ew_voiced))
                em_voiced =  (np.sum(em_integrand) - 1/2*(em_integrand[0]+em_integrand[-1]))
                
                Ew_unvoiced = np.ones(len(Sw))
                numer_Am_unvoiced = Sw * np.conj(Ew_unvoiced)
                denom_Am_unvoiced = Ew_unvoiced * np.conj(Ew_unvoiced)
                numer_integrand = (np.sum(numer_Am_unvoiced) - 1/2*(numer_Am_unvoiced[0]+numer_Am_unvoiced[-1]))
                denom_integrand = (np.sum(denom_Am_unvoiced) - 1/2*(denom_Am_unvoiced[0]+denom_Am_unvoiced[-1]))
                Am_unvoiced = numer_integrand / denom_integrand
                em_integrand_un = (Sw - Am_unvoiced * Ew_unvoiced) * np.conj((Sw - Am_unvoiced * Ew_unvoiced))
                em_unvoiced = (np.sum(em_integrand_un) - 1/2*(em_integrand_un[0]+em_integrand_un[-1]))
                
                if (em_voiced > em_unvoiced):
                    refined_Ams.append(Am_unvoiced)
                    refined_choices.append(0)
                    refined_ems.append(np.absolute(em_unvoiced))           
                                  
                else:
                    refined_Ams.append(Am_voiced)
                    refined_choices.append(1)
                    refined_ems.append(np.absolute(em_voiced))
                    
            current_error_sum = np.sum(refined_ems)
            if(current_error_sum < min_error):
                min_error = current_error_sum
                best_Am = refined_Ams
                best_choices = refined_choices
                best_em = refined_ems
                best_pitch = refined_pitch
            
        Ams.append(best_Am)
        choices.append(best_choices)
        ems.append(best_em)
        error_sum.append(min_error)
        refined_pitch_estimates.append(best_pitch)
        
    Ams = np.array(Ams)
    choices = np.array(choices)
    ems = np.array(ems)
    refined_pitch_estimates = np.array(refined_pitch_estimates)
                                  
    return Ams, choices, ems, refined_pitch_estimates

In [ ]:
# Generates voiced signal

def voiced_signal(Ams, choices, refined_pitch_estimates, samples_per_skip, signal_length):
    voiced_signal = np.zeros(signal_length,dtype = 'complex128')
    
    prev_theta = []
    for f in range(len(refined_pitch_estimates)):
        start_n = f*samples_per_skip
        end_n = (f+1)*samples_per_skip
        for n in range(start_n,end_n):
            current_theta = []
            if f == len(refined_pitch_estimates) - 1:
                max_m = int(refined_pitch_estimates[f]) - 1
            else:
                max_m = max(int(refined_pitch_estimates[f]), int(refined_pitch_estimates[f+1]))-1

            for m in range(max_m):
                
                if f == len(refined_pitch_estimates) - 1:
                    w_0 = 2* np.pi /refined_pitch_estimates[f]
                else:
                    w_f = 2* np.pi /refined_pitch_estimates[f]
                    w_f1 = 2* np.pi /refined_pitch_estimates[f+1]
                    w_0 = (f+1- n/samples_per_skip) * w_f + (n/samples_per_skip - f) * w_f1
                    
                if n == 0:
                    theta_m = (m+1) * w_0
                elif m >= len(prev_theta):
                    theta_m = (m+1) * w_0
                else:
                    theta_m = prev_theta[m] + (m+1) * w_0
                current_theta.append(theta_m)
                
                if f == len(refined_pitch_estimates) - 1:
                    A_m_n = Ams[f][m]

                else:
                    if m >= int(refined_pitch_estimates[f])-1:
                        Am_f = 0
                    elif choices[f][m] == 0:
                        Am_f = 0
                    else:
                        Am_f = Ams[f][m]
                    if m >= int(refined_pitch_estimates[f+1])-1:
                        Am_f1 = 0
                    elif choices[f+1][m] == 0:
                        Am_f1 = 0
                    else:
                        Am_f1 = Ams[f+1][m]
                    A_m_n = (f+1- n/samples_per_skip) * Am_f + (n/samples_per_skip - f) * Am_f1
                    
                voiced_signal[n] += A_m_n * np.cos(theta_m)
                
            prev_theta = current_theta
            
    return voiced_signal

def unvoiced_signal(choices, pitch_estimates, samples_per_frame, samples_per_skip, signal_length, fft_frames, fft_length):
    unvoiced_signal = np.zeros(signal_length,dtype ='complex128')
    
    # reconstruct unvoiced signals between [fK, (f+1)K]
    unvoiced_frames = []    
    frame_num = len(fft_frames)
    for f in range(frame_num):
        unvoiced_bands_fft = np.zeros(fft_length, dtype ='complex128') # fft of unvoiced bands in current frame
        fft_frame = fft_frames[f]
        pitch = pitch_estimates[f]
        w_0 = 2 * np.pi / pitch
        
        for m in range(0,int(pitch)-1):
            
            if choices[f][m] == 0:
                
                center_freq = w_0 * m
                lower_freq = w_0 * (m-1/2)
                upper_freq = w_0 * (m+1/2)

                freq_rate = fft_length / (2 * np.pi)
                a_m = int(lower_freq * freq_rate) + 1
                b_m = min(int(upper_freq * freq_rate),fft_length - 1)
                
                Sw = fft_frame[a_m:b_m+1]
                var = 1/(b_m - a_m) * np.sum(Sw * np.conj(Sw))
                unvoiced_var = 0.5 * np.absolute(var)
                
                for k in range(a_m, b_m + 1):
                    unvoiced_bands_fft[k] = complex(np.random.normal(0,np.sqrt(unvoiced_var)),
                                                    np.random.normal(0,np.sqrt(unvoiced_var)))
        
        unvoiced_bands = np.fft.ifft(unvoiced_bands_fft,fft_length) # unvoiced signal in current frame
        unvoiced_frames.append(unvoiced_bands[0:samples_per_frame])
        
    unvoiced_frames = np.array(unvoiced_frames)
    for f in range(frame_num):
        start_n = f * samples_per_skip
        end_n = (f+1) * samples_per_skip
        for n in range(start_n,end_n):
            if f == frame_num - 1:
                unvoiced_signal[n] = unvoiced_frames[f][n-f*samples_per_skip]
            else:
                curr_unvoiced = (f+1- n/samples_per_skip) * unvoiced_frames[f][n-f*samples_per_skip]
                next_unvoiced = (n/samples_per_skip - f) * unvoiced_frames[f+1][n-(f+1)*samples_per_skip]
                unvoiced_signal[n] = curr_unvoiced + next_unvoiced
                
    return unvoiced_signal

In [ ]:
rate, data = scipy.io.wavfile.read('s5.wav')
data = data.astype('float32')
norm = np.sqrt(np.sum(data*data))
data = data/norm
samples_per_frame = int(25/1000*rate)
samples_per_skip = int(10/1000*rate)
signal_length = data.shape[0]
fft_length = 1024
pitch_candidates = np.arange(20, 91)

frames = extract_frame(data, 
                       rate, 
                       samples_per_frame, 
                       samples_per_skip)

fft_frames = fft_frame(frames, fft_length)

pitch_estimates = estimate_pitch(frames, 
                                 samples_per_frame, 
                                 pitch_candidates)

Ams, choices, errors, refined_pitch_estimates = estimate_amplitude(fft_frames,
                                                           samples_per_frame,
                                                           pitch_estimates,
                                                           fft_length)

In [ ]:
voiced_signal = voiced_signal(Ams, 
                              choices,
                              refined_pitch_estimates,
                              samples_per_skip,
                              signal_length)

unvoiced_signal = unvoiced_signal(choices,
                                  refined_pitch_estimates,
                                  samples_per_frame,
                                  samples_per_skip, 
                                  signal_length, 
                                  fft_frames, 
                                  fft_length)

In [ ]:
scipy.io.wavfile.write("test.wav",rate,np.real(voiced_signal+unvoiced_signal))